# Jupyter Notebook for converting the Go Pass Data


## 1.0 Load in modules

We are using pandas and the numpy libraries to read and process the data.

Then we will use the `rapidfuzz` library to caclulate close matches to the strings so that we can join the information.

Using other popular fuzzy-string matching libraries like `fuzzball` takes over 3 hours to process as the alogrithim used takes the number of records to the `n`th power!! (i.e. comparing 1000 records processes 1000^1000!!!) `rapidfuzz` has a better algorithm that trims this town to 3 - 4 minutes!


In [159]:
import pandas as pd
import numpy as np
from rapidfuzz import process, utils as fuzz_utils

GOOGLE_SHEET_URL = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSWbwrsqF-c---4lfw0LZWymd-f8sy8sLYkXgzh0OyeGATWwrvv7V1Mq5BcApn7F_-WYKP1KXy5shKw/pub?output=csv'

## 2.0 Format Go Pass Data

We will be extracting the following fields:

- District
- School
- Address
- City
- Participating


In [160]:
go_pass_schools = pd.read_csv(GOOGLE_SHEET_URL,
    usecols={'District','School','Address','City','Participating'})
go_pass_schools.columns = ["district","name","address","city","participating"]

go_pass_schools = go_pass_schools.fillna('')
go_pass_schools['original_name'] = go_pass_schools['name']
simple_district = go_pass_schools['district'].str.lower().str.replace(" ","-").str.replace(".","").str.replace('"','')
# go_pass_schools['name'] = go_pass_schools['name'].str.lower().str.replace(" ","-").str.replace(".","").str.replace('"','')+"-"+simple_district
simple_address = go_pass_schools['address'].str.lower().str.replace(" ","-").str.replace(".","").str.replace('"','')
go_pass_schools.loc[(go_pass_schools['name'].str.len() > 1),'name'] = simple_address+"-"+go_pass_schools['name'].str.lower().str.replace(" ","-").str.replace(".","").str.replace('"','')+"-"+simple_district



# go_pass_schools.drop_duplicates(['original_name'])
go_pass_schools

# schools = go_pass_schools['original_name'].unique()
# print(go_pass_schools['original_name'].nunique())

/tmp/ipykernel_30433/312633595.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  simple_district = go_pass_schools['district'].str.lower().str.replace(" ","-").str.replace(".","").str.replace('"','')
/tmp/ipykernel_30433/312633595.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  simple_address = go_pass_schools['address'].str.lower().str.replace(" ","-").str.replace(".","").str.replace('"','')
/tmp/ipykernel_30433/312633595.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  go_pass_schools.loc[(go_pass_schools['na

,district,name,address,city,participating,original_name
0,Centinela Valley,4859-west-el-segundo-blvd-hawthorne-high-centi...,4859 West El Segundo Blvd.,Hawthorne,True,Hawthorne High
1,Centinela Valley,14901-s-inglewood-avenue-lawndale-high-centine...,14901 S. Inglewood Avenue,Lawndale,True,Lawndale High
2,Centinela Valley,4118-west-rosecrans-ave-leuzinger-high-centine...,4118 West Rosecrans Ave.,Lawndale,True,Leuzinger High
3,Centinela Valley,4951-marine-ave-r-k-lloyde-high-centinela-valley,4951 Marine Ave.,Lawndale,True,R. K. Lloyde High
4,Charter,"461-9th-street,-san-pedro-alliance-alice-m-bax...","461 9th Street, San Pedro",San Pedro,True,Alliance Alice M. Baxter College-Ready High Sc...
...,...,...,...,...,...,...
1477,Private,1253-bishops-road-cathedral-high-school-private,1253 Bishops Road,Los Angeles,True,Cathedral High School
1478,Private,6361-santa-monica-blvd-episcopal-school-of-los...,6361 Santa Monica Blvd.,Los Angeles,True,Episcopal School of Los Angeles
1479,Private,9650-zelzah-ave-northpoint-school-private,9650 Zelzah Ave.,Northridge,True,Northpoint School
1480,Santa Monica-Malibu,,,,True,


### Source for schools data

https://www.cde.ca.gov/SchoolDirectory/ExportSelect?simpleSearch=N&address=&city=&counties=&districts=&cdscode=&charter=&magnet=&name=&nps=&search=2&zip=&yearround=&status=1%2C2&types=&order=1&multilingual=&qsc=3549&qdc=3549

July 2022


In [161]:
california_schools_data = "../data/california_schools.csv"

california_schools = pd.read_csv(california_schools_data,
    usecols={'County','Status','District','School','Street Address','Street City'},encoding='latin')
california_schools.columns = ["county","district","name","status","address","city"]

california_schools = california_schools.fillna('')
la_schools_df = california_schools[california_schools["county"] == "Los Angeles"]

print(la_schools_df.shape)
la_schools_df['original_name'] = la_schools_df['name']

la_schools_df.loc[la_schools_df['district'] == "Los Angeles Unified", 'district'] = "LAUSD"
simple_district = la_schools_df['district'].str.lower().str.replace(" ","-").str.replace(".","").str.replace('"','')
simple_address = la_schools_df['address'].str.lower().str.replace(" ","-").str.replace(".","").str.replace('"','')

la_schools_df['simple_name'] = simple_address+"-"+la_schools_df['name'].str.lower().str.replace(" ","-").str.replace(".","").str.replace('"','')+"-"+simple_district


la_schools_df

/tmp/ipykernel_30433/588626390.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  la_schools_df['original_name'] = la_schools_df['name']
/tmp/ipykernel_30433/588626390.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  simple_district = la_schools_df['district'].str.lower().str.replace(" ","-").str.replace(".","").str.replace('"','')


(5829, 6)


/tmp/ipykernel_30433/588626390.py:15: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  simple_address = la_schools_df['address'].str.lower().str.replace(" ","-").str.replace(".","").str.replace('"','')
/tmp/ipykernel_30433/588626390.py:17: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  la_schools_df['simple_name'] = simple_address+"-"+la_schools_df['name'].str.lower().str.replace(" ","-").str.replace(".","").str.replace('"','')+"-"+simple_district
/tmp/ipykernel_30433/588626390.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pan

,county,district,name,status,address,city,original_name,simple_name
4409,Los Angeles,ABC Unified,ABC Adult,Active,12254 Cuesta Dr.,Cerritos,ABC Adult,12254-cuesta-dr-abc-adult-abc-unified
4410,Los Angeles,ABC Unified,ABC Evening High School,Closed,16800 Shoemaker Ave.,Cerritos,ABC Evening High School,16800-shoemaker-ave-abc-evening-high-school-ab...
4411,Los Angeles,ABC Unified,ABC Secondary (Alternative),Active,16534 South Carmenita Rd.,Cerritos,ABC Secondary (Alternative),16534-south-carmenita-rd-abc-secondary-(altern...
4412,Los Angeles,ABC Unified,Accelerated Christian Academy,Closed,Information Redacted,Lakewood,Accelerated Christian Academy,information-redacted-accelerated-christian-aca...
4413,Los Angeles,ABC Unified,Aloha Elementary,Active,11737 East 214th St.,Lakewood,Aloha Elementary,11737-east-214th-st-aloha-elementary-abc-unified
...,...,...,...,...,...,...,...,...
10233,Los Angeles,Wiseburn Unified,Juan De Anza Elementary,Active,12110 Hindry Ave.,Hawthorne,Juan De Anza Elementary,12110-hindry-ave-juan-de-anza-elementary-wiseb...
10234,Los Angeles,Wiseburn Unified,Malaga Cove School - Success Learning Center,Closed,300 Paseo Del Mar,Palos Verdes Estates,Malaga Cove School - Success Learning Center,300-paseo-del-mar-malaga-cove-school---success...
10235,Los Angeles,Wiseburn Unified,Richard Henry Dana Middle,Active,5504 West 135th St.,Hawthorne,Richard Henry Dana Middle,5504-west-135th-st-richard-henry-dana-middle-w...
10236,Los Angeles,Wiseburn Unified,RISE High,Closed,13500 Aviation Blvd.,Hawthorne,RISE High,13500-aviation-blvd-rise-high-wiseburn-unified


In [162]:
unique_schools = go_pass_schools.drop_duplicates(subset=['name'])
print(go_pass_schools.shape)
print(unique_schools.shape)

print(go_pass_schools.shape[0] - unique_schools.shape[0])

# unique_schools.groupby('original_name')
# a = unique_schools.groupby(['original_name','district']).nunique()
# unique_schools.groupby(['original_name']).agg(['count'])
# a
# unique_schools.value_counts(['original_name'])
# unique_schools

(1482, 6)
(1477, 6)
5


In [163]:
def fuzzy_merge(baseFrame, compareFrame, baseKey, compareKey, threshold=80, limit=0, how='left'):
    s_mapping = {x: fuzz_utils.default_process(x) for x in compareFrame[compareKey]}

    m1 = baseFrame[baseKey].apply(lambda x: process.extract(
      fuzz_utils.default_process(x), s_mapping, limit=limit, score_cutoff=threshold, processor=None
    ))
    baseFrame['Match'] = m1

    m2 = baseFrame['Match'].apply(lambda x: ', '.join(i[2] for i in x))
    baseFrame['name'] = m2.replace("",np.nan)

    return baseFrame.merge(compareFrame, left_on='name', right_on=compareKey, how=how)


# go_pass_schools_with_names = unique_schools['name'].str.len() > 1
# go_pass_schools_with_names = unique_schools.loc[unique_schools['name'].str.len() > 1]
# go_pass_schools_no_names = go_pass_schools.loc[go_pass_schools['name'].str.len() < 1]
merged_df = fuzzy_merge(go_pass_schools, la_schools_df, 'name', 'simple_name',how='left')
# go_pass_schools_with_names
merged_df


,district_x,name_x,address_x,city_x,participating,original_name_x,Match,county,district_y,name_y,status,address_y,city_y,original_name_y,simple_name
0,Centinela Valley,4859-west-el-segundo-blvd-hawthorne-high-centi...,4859 West El Segundo Blvd.,Hawthorne,True,Hawthorne High,[(4859 west el segundo blvd hawthorne high cen...,Los Angeles,Centinela Valley Union High,Hawthorne High,Active,4859 West El Segundo Blvd.,Hawthorne,Hawthorne High,4859-west-el-segundo-blvd-hawthorne-high-centi...
1,Centinela Valley,14901-south-inglewood-ave-lawndale-high-centin...,14901 S. Inglewood Avenue,Lawndale,True,Lawndale High,[(14901 south inglewood ave lawndale high cent...,Los Angeles,Centinela Valley Union High,Lawndale High,Active,14901 South Inglewood Ave.,Lawndale,Lawndale High,14901-south-inglewood-ave-lawndale-high-centin...
2,Centinela Valley,4118-west-rosecrans-ave-leuzinger-high-centine...,4118 West Rosecrans Ave.,Lawndale,True,Leuzinger High,[(4118 west rosecrans ave leuzinger high centi...,Los Angeles,Centinela Valley Union High,Leuzinger High,Active,4118 West Rosecrans Ave.,Lawndale,Leuzinger High,4118-west-rosecrans-ave-leuzinger-high-centine...
3,Centinela Valley,4951-marine-ave-r-k-lloyde-high-centinela-vall...,4951 Marine Ave.,Lawndale,True,R. K. Lloyde High,[(4951 marine ave r k lloyde high centinela va...,Los Angeles,Centinela Valley Union High,R. K. Lloyde High,Active,4951 Marine Ave.,Lawndale,R. K. Lloyde High,4951-marine-ave-r-k-lloyde-high-centinela-vall...
4,Charter,16800-shoemaker-ave-abc-evening-high-school-ab...,"461 9th Street, San Pedro",San Pedro,True,Alliance Alice M. Baxter College-Ready High Sc...,[(16800 shoemaker ave abc evening high school ...,Los Angeles,ABC Unified,ABC Evening High School,Closed,16800 Shoemaker Ave.,Cerritos,ABC Evening High School,16800-shoemaker-ave-abc-evening-high-school-ab...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1485,Private,1253-bishops-rd-cathedral-high-school-lausd,1253 Bishops Road,Los Angeles,True,Cathedral High School,"[(1253 bishops rd cathedral high school lausd,...",Los Angeles,LAUSD,Cathedral High School,Active,1253 Bishops Rd.,Los Angeles,Cathedral High School,1253-bishops-rd-cathedral-high-school-lausd
1486,Private,6361-santa-monica-blvd-the-episcopal-school-of...,6361 Santa Monica Blvd.,Los Angeles,True,Episcopal School of Los Angeles,[(6361 santa monica blvd the episcopal school ...,Los Angeles,LAUSD,The Episcopal School of Los Angeles,Active,6361 Santa Monica Blvd.,Los Angeles,The Episcopal School of Los Angeles,6361-santa-monica-blvd-the-episcopal-school-of...
1487,Private,information-redacted-aspen-ridge-preparatory-s...,9650 Zelzah Ave.,Northridge,True,Northpoint School,[(information redacted aspen ridge preparatory...,Los Angeles,Acton-Agua Dulce Unified,Aspen Ridge Preparatory School,Closed,Information Redacted,Acton,Aspen Ridge Preparatory School,information-redacted-aspen-ridge-preparatory-s...
1488,Santa Monica-Malibu,NaN,,,True,,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [164]:
# merged_df.append(go_pass_schools_no_names)
merged_df

,district_x,name_x,address_x,city_x,participating,original_name_x,Match,county,district_y,name_y,status,address_y,city_y,original_name_y,simple_name
0,Centinela Valley,4859-west-el-segundo-blvd-hawthorne-high-centi...,4859 West El Segundo Blvd.,Hawthorne,True,Hawthorne High,[(4859 west el segundo blvd hawthorne high cen...,Los Angeles,Centinela Valley Union High,Hawthorne High,Active,4859 West El Segundo Blvd.,Hawthorne,Hawthorne High,4859-west-el-segundo-blvd-hawthorne-high-centi...
1,Centinela Valley,14901-south-inglewood-ave-lawndale-high-centin...,14901 S. Inglewood Avenue,Lawndale,True,Lawndale High,[(14901 south inglewood ave lawndale high cent...,Los Angeles,Centinela Valley Union High,Lawndale High,Active,14901 South Inglewood Ave.,Lawndale,Lawndale High,14901-south-inglewood-ave-lawndale-high-centin...
2,Centinela Valley,4118-west-rosecrans-ave-leuzinger-high-centine...,4118 West Rosecrans Ave.,Lawndale,True,Leuzinger High,[(4118 west rosecrans ave leuzinger high centi...,Los Angeles,Centinela Valley Union High,Leuzinger High,Active,4118 West Rosecrans Ave.,Lawndale,Leuzinger High,4118-west-rosecrans-ave-leuzinger-high-centine...
3,Centinela Valley,4951-marine-ave-r-k-lloyde-high-centinela-vall...,4951 Marine Ave.,Lawndale,True,R. K. Lloyde High,[(4951 marine ave r k lloyde high centinela va...,Los Angeles,Centinela Valley Union High,R. K. Lloyde High,Active,4951 Marine Ave.,Lawndale,R. K. Lloyde High,4951-marine-ave-r-k-lloyde-high-centinela-vall...
4,Charter,16800-shoemaker-ave-abc-evening-high-school-ab...,"461 9th Street, San Pedro",San Pedro,True,Alliance Alice M. Baxter College-Ready High Sc...,[(16800 shoemaker ave abc evening high school ...,Los Angeles,ABC Unified,ABC Evening High School,Closed,16800 Shoemaker Ave.,Cerritos,ABC Evening High School,16800-shoemaker-ave-abc-evening-high-school-ab...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1485,Private,1253-bishops-rd-cathedral-high-school-lausd,1253 Bishops Road,Los Angeles,True,Cathedral High School,"[(1253 bishops rd cathedral high school lausd,...",Los Angeles,LAUSD,Cathedral High School,Active,1253 Bishops Rd.,Los Angeles,Cathedral High School,1253-bishops-rd-cathedral-high-school-lausd
1486,Private,6361-santa-monica-blvd-the-episcopal-school-of...,6361 Santa Monica Blvd.,Los Angeles,True,Episcopal School of Los Angeles,[(6361 santa monica blvd the episcopal school ...,Los Angeles,LAUSD,The Episcopal School of Los Angeles,Active,6361 Santa Monica Blvd.,Los Angeles,The Episcopal School of Los Angeles,6361-santa-monica-blvd-the-episcopal-school-of...
1487,Private,information-redacted-aspen-ridge-preparatory-s...,9650 Zelzah Ave.,Northridge,True,Northpoint School,[(information redacted aspen ridge preparatory...,Los Angeles,Acton-Agua Dulce Unified,Aspen Ridge Preparatory School,Closed,Information Redacted,Acton,Aspen Ridge Preparatory School,information-redacted-aspen-ridge-preparatory-s...
1488,Santa Monica-Malibu,NaN,,,True,,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [165]:
merged_df['full_name'] = merged_df['original_name_x'] + ' (' + merged_df['district_x'] + ')'
merged_df['score'] = merged_df['Match'].astype('string').str.split(",").str[1]

unique_name = merged_df['original_name_x'] + ' (' + merged_df['district_y'] + ')'
merged_df['duped'] = merged_df.duplicated(['original_name_x'],keep=False)

merged_df.loc[merged_df['duped'] == False, 'full_name_some'] = merged_df['original_name_x']

merged_df.loc[merged_df['duped'] == True, 'full_name_some'] = merged_df['original_name_x'] + ' (' + merged_df['district_y'] + ')'
# merged_df.loc[(merged_df['name'].str.len() < 1) & (merged_df['duped'] == True), 'full_name_some'] = merged_df['district_x']
merged_df

,district_x,name_x,address_x,city_x,participating,original_name_x,Match,county,district_y,name_y,status,address_y,city_y,original_name_y,simple_name,full_name,score,duped,full_name_some
0,Centinela Valley,4859-west-el-segundo-blvd-hawthorne-high-centi...,4859 West El Segundo Blvd.,Hawthorne,True,Hawthorne High,[(4859 west el segundo blvd hawthorne high cen...,Los Angeles,Centinela Valley Union High,Hawthorne High,Active,4859 West El Segundo Blvd.,Hawthorne,Hawthorne High,4859-west-el-segundo-blvd-hawthorne-high-centi...,Hawthorne High (Centinela Valley),95.0,False,Hawthorne High
1,Centinela Valley,14901-south-inglewood-ave-lawndale-high-centin...,14901 S. Inglewood Avenue,Lawndale,True,Lawndale High,[(14901 south inglewood ave lawndale high cent...,Los Angeles,Centinela Valley Union High,Lawndale High,Active,14901 South Inglewood Ave.,Lawndale,Lawndale High,14901-south-inglewood-ave-lawndale-high-centin...,Lawndale High (Centinela Valley),86.53465346534652,False,Lawndale High
2,Centinela Valley,4118-west-rosecrans-ave-leuzinger-high-centine...,4118 West Rosecrans Ave.,Lawndale,True,Leuzinger High,[(4118 west rosecrans ave leuzinger high centi...,Los Angeles,Centinela Valley Union High,Leuzinger High,Active,4118 West Rosecrans Ave.,Lawndale,Leuzinger High,4118-west-rosecrans-ave-leuzinger-high-centine...,Leuzinger High (Centinela Valley),95.0,False,Leuzinger High
3,Centinela Valley,4951-marine-ave-r-k-lloyde-high-centinela-vall...,4951 Marine Ave.,Lawndale,True,R. K. Lloyde High,[(4951 marine ave r k lloyde high centinela va...,Los Angeles,Centinela Valley Union High,R. K. Lloyde High,Active,4951 Marine Ave.,Lawndale,R. K. Lloyde High,4951-marine-ave-r-k-lloyde-high-centinela-vall...,R. K. Lloyde High (Centinela Valley),95.0,False,R. K. Lloyde High
4,Charter,16800-shoemaker-ave-abc-evening-high-school-ab...,"461 9th Street, San Pedro",San Pedro,True,Alliance Alice M. Baxter College-Ready High Sc...,[(16800 shoemaker ave abc evening high school ...,Los Angeles,ABC Unified,ABC Evening High School,Closed,16800 Shoemaker Ave.,Cerritos,ABC Evening High School,16800-shoemaker-ave-abc-evening-high-school-ab...,Alliance Alice M. Baxter College-Ready High Sc...,85.5,False,Alliance Alice M. Baxter College-Ready High Sc...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1485,Private,1253-bishops-rd-cathedral-high-school-lausd,1253 Bishops Road,Los Angeles,True,Cathedral High School,"[(1253 bishops rd cathedral high school lausd,...",Los Angeles,LAUSD,Cathedral High School,Active,1253 Bishops Rd.,Los Angeles,Cathedral High School,1253-bishops-rd-cathedral-high-school-lausd,Cathedral High School (Private),86.66666666666667,False,Cathedral High School
1486,Private,6361-santa-monica-blvd-the-episcopal-school-of...,6361 Santa Monica Blvd.,Los Angeles,True,Episcopal School of Los Angeles,[(6361 santa monica blvd the episcopal school ...,Los Angeles,LAUSD,The Episcopal School of Los Angeles,Active,6361 Santa Monica Blvd.,Los Angeles,The Episcopal School of Los Angeles,6361-santa-monica-blvd-the-episcopal-school-of...,Episcopal School of Los Angeles (Private),88.88888888888889,False,Episcopal School of Los Angeles
1487,Private,information-redacted-aspen-ridge-preparatory-s...,9650 Zelzah Ave.,Northridge,True,Northpoint School,[(information redacted aspen ridge preparatory...,Los Angeles,Acton-Agua Dulce Unified,Aspen Ridge Preparatory School,Closed,Information Redacted,Acton,Aspen Ridge Preparatory School,information-redacted-aspen-ridge-preparatory-s...,Northpoint School (Private),85.5,False,Northpoint School
1488,Santa Monica-Malibu,NaN,,,True,,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(Santa Monica-Malibu),NaN,True,NaN


In [166]:
final_columns = {
    "full_name":"school_name",
    "full_name_some":"school_name_with_some_districts_attached",
}

final_df = merged_df[["district_x","district_y","original_name_x","original_name_y","status","full_name","full_name_some","participating","address_x","address_y","city_x","city_y","score","duped"]]
# final_df = merged_df["full_name","participating","score","full_name_some","duped"]
final_df.rename(inplace=True, columns=final_columns)
final_df.index.names = ['id']
final_df

/tmp/ipykernel_30433/2893358900.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.rename(inplace=True, columns=final_columns)


,district_x,district_y,original_name_x,original_name_y,status,school_name,school_name_with_some_districts_attached,participating,address_x,address_y,city_x,city_y,score,duped
id,,,,,,,,,,,,,,
0,Centinela Valley,Centinela Valley Union High,Hawthorne High,Hawthorne High,Active,Hawthorne High (Centinela Valley),Hawthorne High,True,4859 West El Segundo Blvd.,4859 West El Segundo Blvd.,Hawthorne,Hawthorne,95.0,False
1,Centinela Valley,Centinela Valley Union High,Lawndale High,Lawndale High,Active,Lawndale High (Centinela Valley),Lawndale High,True,14901 S. Inglewood Avenue,14901 South Inglewood Ave.,Lawndale,Lawndale,86.53465346534652,False
2,Centinela Valley,Centinela Valley Union High,Leuzinger High,Leuzinger High,Active,Leuzinger High (Centinela Valley),Leuzinger High,True,4118 West Rosecrans Ave.,4118 West Rosecrans Ave.,Lawndale,Lawndale,95.0,False
3,Centinela Valley,Centinela Valley Union High,R. K. Lloyde High,R. K. Lloyde High,Active,R. K. Lloyde High (Centinela Valley),R. K. Lloyde High,True,4951 Marine Ave.,4951 Marine Ave.,Lawndale,Lawndale,95.0,False
4,Charter,ABC Unified,Alliance Alice M. Baxter College-Ready High Sc...,ABC Evening High School,Closed,Alliance Alice M. Baxter College-Ready High Sc...,Alliance Alice M. Baxter College-Ready High Sc...,True,"461 9th Street, San Pedro",16800 Shoemaker Ave.,San Pedro,Cerritos,85.5,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1485,Private,LAUSD,Cathedral High School,Cathedral High School,Active,Cathedral High School (Private),Cathedral High School,True,1253 Bishops Road,1253 Bishops Rd.,Los Angeles,Los Angeles,86.66666666666667,False
1486,Private,LAUSD,Episcopal School of Los Angeles,The Episcopal School of Los Angeles,Active,Episcopal School of Los Angeles (Private),Episcopal School of Los Angeles,True,6361 Santa Monica Blvd.,6361 Santa Monica Blvd.,Los Angeles,Los Angeles,88.88888888888889,False
1487,Private,Acton-Agua Dulce Unified,Northpoint School,Aspen Ridge Preparatory School,Closed,Northpoint School (Private),Northpoint School,True,9650 Zelzah Ave.,Information Redacted,Northridge,Acton,85.5,False


In [167]:
final_df.to_csv('../data/go_pass_schools_merged_with_california_dataset.csv')